
# 08 — Real-World RAG Patterns (Finance / Health / Legal)

This notebook contains simple, runnable skeletons for:
- Finance Research Assistant
- ENT Guidelines Assistant
- Contract Analyzer Copilot

Each section shows:
- Minimal RAG setup
- Domain-shaped retrieval



## 1. Finance Research Assistant (RAG Skeleton)

Domain: financial education / filings / research snippets.


In [ ]:

!pip install langchain langchain-openai chromadb langchain-community --quiet

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

with open("finance_corpus.txt", "w") as f:
    f.write("An ETF is an Exchange Traded Fund that tracks an index.\n")
    f.write("Dollar-cost averaging is an investment strategy where you invest a fixed amount regularly.\n")

loader = TextLoader("finance_corpus.txt")
finance_docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
finance_chunks = splitter.split_documents(finance_docs)

finance_emb = OpenAIEmbeddings()
finance_db = Chroma.from_documents(finance_chunks, finance_emb)
finance_llm = ChatOpenAI(model="gpt-4o-mini")

def finance_rag(question: str) -> str:
    docs = finance_db.similarity_search(question, k=3)
    ctx = "\n".join(d.page_content for d in docs)
    prompt = f"Finance context (educational only):\n{ctx}\n\nQuestion: {question}\nAnswer clearly for a beginner:"
    return finance_llm.invoke(prompt).content

print(finance_rag("What is an ETF?"))



## 2. ENT Guidelines Assistant (RAG Skeleton)

Domain: ENT guideline-like snippets for demonstration.


In [ ]:

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

with open("ent_guidelines.txt", "w") as f:
    f.write("Chronic otitis media: First-line management includes assessment of hearing and infection control.\n")
    f.write("Acute sinusitis: Watchful waiting may be appropriate in mild cases, antibiotics reserved for severe or persistent symptoms.\n")

ent_loader = TextLoader("ent_guidelines.txt")
ent_docs = ent_loader.load()

ent_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
ent_chunks = ent_splitter.split_documents(ent_docs)

ent_emb = OpenAIEmbeddings()
ent_db = Chroma.from_documents(ent_chunks, ent_emb)
ent_llm = ChatOpenAI(model="gpt-4o-mini")

def ent_guideline_rag(question: str) -> str:
    docs = ent_db.similarity_search(question, k=3)
    ctx = "\n".join(d.page_content for d in docs)
    prompt = f"""You are an ENT guideline assistant.
Use ONLY the context below. Do not give diagnosis or personalized advice.

Context:
{ctx}

Question: {question}

Answer with a neutral guideline-style summary and mention that this is not medical advice.
"""
    return ent_llm.invoke(prompt).content

print(ent_guideline_rag("How is chronic otitis media managed?"))



## 3. Contract Analyzer Copilot (RAG Skeleton)

Domain: contract clauses (simulated NDA/MSA-style text).


In [ ]:

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

with open("contracts_corpus.txt", "w") as f:
    f.write("Confidentiality: Each party agrees to keep all confidential information in strict confidence.\n")
    f.write("Limitation of liability: Liability shall be limited to direct damages and capped at the fees paid in the last 12 months.\n")

contract_loader = TextLoader("contracts_corpus.txt")
contract_docs = contract_loader.load()

contract_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
contract_chunks = contract_splitter.split_documents(contract_docs)

contract_emb = OpenAIEmbeddings()
contract_db = Chroma.from_documents(contract_chunks, contract_emb)
contract_llm = ChatOpenAI(model="gpt-4o-mini")

def contract_rag(question: str) -> str:
    docs = contract_db.similarity_search(question, k=3)
    ctx = "\n".join(d.page_content for d in docs)
    prompt = f"""You are a contract analysis assistant for legal professionals.
Use ONLY the contractual context below.

Context:
{ctx}

Question: {question}

Provide a neutral summary of the relevant clause(s) and clearly say this is not legal advice.
"""
    return contract_llm.invoke(prompt).content

print(contract_rag("What does the limitation of liability clause say?"))



## End of Notebook 08  
Finance / ENT / Legal RAG skeletons are ready for extension.
